# Post-processing DayCent corn stover regional simulation results
This Jupyter Notebook is designed to facilitate post-processing and analysis of sets of raw DayCent results from a regional scale simulation. For more information, contact author [John Field](https://johnlfield.weebly.com/) at <john.L.field@gmail.com>

## DayCent background
DayCent is a process-based model that simulates agro-ecosystem net primary production, soil organic matter dynamics, and nitrogen (N) cycling and trace gas emissions. DayCent is a daily-timestep version of the older CENTURY model. Both models were created and are currently maintained at the Colorado State University [Natural Resource Ecology Laboratory](https://www.nrel.colostate.edu/) (CSU-NREL), and source code is available upon request. 
![Alt text](DayCent.png)

DayCent model homepage:  [https://www2.nrel.colostate.edu/projects/daycent/](https://www2.nrel.colostate.edu/projects/daycent/)

In bioenergy sustainability studies, DayCent is typically used to estimate:
* biomass yields
* annual emissions of nitrous oxide (N2O), a potent greenhouse gas (GHG) generated in soils from synthetic and organic nitrogen fertilizer
* changes in soil organic carbon (SOC) levels over time

## Regional simulation workflow
The primary spatial data inputs to DayCent are:
* soil texture as a function of depth
* historic daily weather (Tmin, Tmax, precip)

Our DayCent spatial modeling workflow is based on a national-scale GIS database of current land use ([NLCD](https://www.mrlc.gov/national-land-cover-database-nlcd-2016)), soil ([SSURGO](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/survey/?cid=nrcs142p2_053627)), and weather ([NARR](https://www.ncdc.noaa.gov/data-access/model-data/model-datasets/north-american-regional-reanalysis-narr)) data layers housed at CSU-NREL. The python-based workflow consists of a collection of scripts that perform the following:
1. Selection of area to be simulated, specified based on current land cover and/or land biophysical factors (i.e., soil texutre, slope, land capability class rating, etc.)
2. Determination of individual unique DayCent model runs (i.e., **"strata"**) necessary to cover the heterogenity of soils and climate across the simulation area
3. Parallel execution of simulations on the CSU-NREL computing cluster
4. Results analysis and mapping (this routine)


In [1]:
# import the necessary modules
import constants
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import pandas as pd
import plotly as py
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

## Loading data
The code below loads partially-analyzed DayCent simulation results at the county scale into Pandas dataframes for furhter analysis. Results are included for four different management scenarios that were simulated:
* no stover removal (G, i.e., grain harvest only)
* 25% stover harvest (G25S)
* 50% stover harvest (G50S)
* 75% stover harvest (G75S)

Simulation results are spread across multiple .csv-format results files as follows:
* corn–soy area per county (area_fips_data.csv)
* annual simulated corn yields per county (corn_yield_year_county.csv)
* annual simulated soy yields per county (soybean_yield_year_county.csv)
* annual simulated stover yields per county (Stover_yield_year_county.csv)
* annual simulated SOC levels per county (SOC_year_county.csv)
* annual simulated N2O emissions per county (N2O_year_county.csv)

In [2]:
area_df = pd.read_csv("area_fips_data.csv", usecols=[1, 2])
corn_df = pd.read_csv("corn_yield_year_county.csv", usecols=[1, 2, 3, 4])
soy_df = pd.read_csv("soybean_yield_year_county.csv", usecols=[1, 2, 3, 4])
stover_df = pd.read_csv("Stover_yield_year_county.csv", usecols=[1, 2, 3, 4])
soc_df = pd.read_csv("SOC_year_county.csv", usecols=[1, 2, 3, 4])
n2o_df = pd.read_csv("N2O_year_county.csv", usecols=[1, 2, 3, 4])
soc_df

,fips,simyear,stover_removal,SOC_20cm_g_m2
0,17001,2011,G,6163.79256
1,17001,2011,G25S,6163.65078
2,17001,2011,G50S,6163.45078
3,17001,2011,G75S,6163.21790
4,17001,2012,G,6238.24324
...,...,...,...,...
72595,55139,2039,G75S,4482.88127
72596,55139,2040,G,5133.82478
72597,55139,2040,G25S,4911.76100
72598,55139,2040,G50S,4689.96280


## Unit conversions
In the following operations I'm updating to more formal units using the conversion factors in constants.py. I keep track of the updated units in the column names. For simplicity, after each conversion, I drop the original data in now-obsolete units.

In [3]:
area_df["area_ha"] = area_df["area_acres"] * constants.ha_per_ACRE
area_df.drop(columns=["area_acres"], inplace=True)

corn_df["corn_yield_Mg_ha-1"] = corn_df["grainyield_bu_ac"] * ((constants.kg_per_bu_CORN * 0.001) / constants.ha_per_ACRE)
corn_df.drop(columns=["grainyield_bu_ac"], inplace=True)

soy_df["soy_yield_Mg_ha-1"] = soy_df["grainyield_bu_ac"] * ((constants.kg_per_bu_SOY * 0.001) / constants.ha_per_ACRE)
soy_df.drop(columns=["grainyield_bu_ac"], inplace=True)

stover_df["stover_yield_Mg_ha-1"] = stover_df["stover_dryyield_kgha"] * 0.001
stover_df.drop(columns=["stover_dryyield_kgha"], inplace=True)

soc_df["SOC_MgC_ha-1"] = soc_df["SOC_20cm_g_m2"] * constants.g_m2_to_Mg_ha
soc_df.drop(columns=["SOC_20cm_g_m2"], inplace=True)

n2o_df["N2O_MgN_ha-1"] = n2o_df["N2O_gN_m2"] * constants.g_m2_to_Mg_ha
n2o_df.drop(columns=["N2O_gN_m2"], inplace=True)

soc_df

,fips,simyear,stover_removal,SOC_MgC_ha-1
0,17001,2011,G,61.637926
1,17001,2011,G25S,61.636508
2,17001,2011,G50S,61.634508
3,17001,2011,G75S,61.632179
4,17001,2012,G,62.382432
...,...,...,...,...
72595,55139,2039,G75S,44.828813
72596,55139,2040,G,51.338248
72597,55139,2040,G25S,49.117610
72598,55139,2040,G50S,46.899628


## Computing annual SOC differences
I implemented this by ordering the data by fips|treatment|time, computing row differences, and then dropping the first year of each series (which reflects a difference between treatments instead of between years).

In [4]:
soc_df = soc_df.sort_values(["fips", "stover_removal", "simyear"])
soc_df["dSOC_MgC_ha-1"] = soc_df["SOC_MgC_ha-1"].diff()
soc_df = soc_df[(soc_df["simyear"] != 2011)]
soc_df

,fips,simyear,stover_removal,SOC_MgC_ha-1,dSOC_MgC_ha-1
4,17001,2012,G,62.382432,0.744507
8,17001,2013,G,62.883798,0.501366
12,17001,2014,G,63.421484,0.537686
16,17001,2015,G,63.768391,0.346907
20,17001,2016,G,64.530508,0.762116
...,...,...,...,...,...
72583,55139,2036,G75S,44.601332,-0.167837
72587,55139,2037,G75S,44.624665,0.023333
72591,55139,2038,G75S,44.583367,-0.041298
72595,55139,2039,G75S,44.828813,0.245446


## Data aggregation & merges
Here I calculate mean yields, N2O emissions & SOC changes over the full course of the simulation, and then merge all results together into a single data frame. Note that the aggregation has to come before the merges, since corn & soy are harvested in alternate years and thus cannot be merged on 'simyear'. Also, note that all year 2011 yield results are dropped on merge, since there are no dSOC results for that year.

In [5]:
corn_df = corn_df[["fips", "stover_removal", "corn_yield_Mg_ha-1"]].groupby(["fips", "stover_removal"]).mean()
soy_df = soy_df[["fips", "stover_removal", "soy_yield_Mg_ha-1"]].groupby(["fips", "stover_removal"]).mean()
stover_df = stover_df[["fips", "stover_removal", "stover_yield_Mg_ha-1"]].groupby(["fips", "stover_removal"]).mean()
soc_df = soc_df[["fips", "stover_removal", "dSOC_MgC_ha-1"]].groupby(["fips", "stover_removal"]).mean()
n2o_df = n2o_df[["fips", "stover_removal", "N2O_MgN_ha-1"]].groupby(["fips", "stover_removal"]).mean()

df = pd.merge(corn_df, soy_df, on=['fips', 'stover_removal'])
df = pd.merge(df, stover_df, on=['fips', 'stover_removal'])
df = pd.merge(df, soc_df, on=['fips', 'stover_removal'])
df = pd.merge(df, n2o_df, on=['fips', 'stover_removal'])
df.reset_index(inplace=True)
df

,fips,stover_removal,corn_yield_Mg_ha-1,soy_yield_Mg_ha-1,stover_yield_Mg_ha-1,dSOC_MgC_ha-1,N2O_MgN_ha-1
0,17001,G,12.167596,3.781892,0.000000,0.446482,0.003555
1,17001,G25S,12.163762,3.792497,3.171844,0.319613,0.003483
2,17001,G50S,12.156676,3.801587,6.340456,0.192506,0.003364
3,17001,G75S,12.139374,3.811355,9.499163,0.060851,0.003311
4,17003,G,10.814741,3.704946,0.000000,0.251531,0.003087
...,...,...,...,...,...,...,...
2415,55133,G75S,10.904004,3.360029,8.453953,0.125345,0.003200
2416,55139,G,11.163102,3.514759,0.000000,0.202491,0.002840
2417,55139,G25S,11.153444,3.516110,2.979155,0.126011,0.002814
2418,55139,G50S,11.143157,3.517085,5.952131,0.049734,0.002730


## Pivot
This operation re-shapes the data such that the results for different management treatments (e.g., G, G25S, etc.) are shown in different columns instead of different rows. 

In [13]:
pivoted_df = df.pivot(index='fips', columns='stover_removal')
pivoted_df

corn_yield_Mg_ha-1                                   \
stover_removal                  G       G25S       G50S       G75S   
fips                                                                 
17001                   12.167596  12.163762  12.156676  12.139374   
17003                   10.814741  10.813140  10.809035  10.801844   
17005                    9.894504   9.880053   9.861649   9.835862   
17007                   12.993146  12.990750  12.987980  12.983564   
17009                   12.291974  12.290263  12.287517  12.282756   
...                           ...        ...        ...        ...   
55109                   11.909724  11.910318  11.911028  11.911382   
55127                   12.638424  12.639205  12.638939  12.638789   
55131                   11.131045  11.122610  11.112415  11.099982   
55133                   10.910526  10.910888  10.907084  10.904004   
55139                   11.163102  11.153444  11.143157  11.131130   

               soy_yield_Mg_ha-1                                \
stover_removal                 G      G25S      G50S      G75S   
fips                                                             
17001                   3.781892  3.792497  3.801587  3.811355   
17003                   3.704946  3.712323  3.719258  3.724909   
17005                   2.916293  2.916226  2.914557  2.910148   
17007                   4.057188  4.065315  4.072851  4.080442   
17009                   3.875002  3.877756  3.879352  3.881847   
...                          ...       ...       ...       ...   
55109                   3.501943  3.506605  3.511580  3.515523   
55127                   3.947402  3.955162  3.962666  3.970324   
55131                   3.384579  3.387988  3.392168  3.395979   
55133                   3.309958  3.326798  3.343789  3.360029   
55139                   3.514759  3.516110  3.517085  3.517293   

               stover_yield_Mg_ha-1                                \
stover_removal                    G      G25S      G50S      G75S   
fips                                                                
17001                           0.0  3.171844  6.340456  9.499163   
17003                           0.0  2.443303  4.884002  7.320607   
17005                           0.0  2.733217  5.452560  8.150314   
17007                           0.0  3.150525  6.299216  9.445878   
17009                           0.0  3.184914  6.367691  9.548746   
...                             ...       ...       ...       ...   
55109                           0.0  2.903239  5.806783  8.709807   
55127                           0.0  3.040777  6.081192  9.120394   
55131                           0.0  2.889806  5.773001  8.646245   
55133                           0.0  2.821860  5.640000  8.453953   
55139                           0.0  2.979155  5.952131  8.917132   

               dSOC_MgC_ha-1                               N2O_MgN_ha-1  \
stover_removal             G      G25S      G50S      G75S            G   
fips                                                                      
17001               0.446482  0.319613  0.192506  0.060851     0.003555   
17003               0.251531  0.171861  0.091403  0.008378     0.003087   
17005               0.200798  0.125962  0.050596 -0.026640     0.002504   
17007               0.319291  0.212863  0.107129 -0.001549     0.003596   
17009               0.258389  0.161073  0.064415 -0.035095     0.003386   
...                      ...       ...       ...       ...          ...   
55109               0.246852  0.161549  0.076714 -0.010572     0.002712   
55127               0.325701  0.225243  0.125056  0.022447     0.003579   
55131               0.263484  0.177798  0.092119  0.004928     0.002936   
55133               0.484862  0.366276  0.247172  0.125345     0.003167   
55139               0.202491  0.126011  0.049734 -0.028172     0.002840   

                                              
stover_removal      G25S      G50S      G75S

In [17]:
pivoted_df.index
pivoted_df['stover_yield_Mg_ha-1']['G25S']

fips
17001    3.171844
17003    2.443303
17005    2.733217
17007    3.150525
17009    3.184914
           ...   
55109    2.903239
55127    3.040777
55131    2.889806
55133    2.821860
55139    2.979155
Name: G25S, Length: 605, dtype: float64

## Remaining operations
* compute net CO2e biogenic GHG footprint
* pivot and calculate relative results for each scenario
* save select results to file

## Mapping our results
Below we use the Plotly module "choropleth" tool to map select county-scale results. The fips_mapping() function helps to standardize the formatting, color scheme, and scale for each map.  

In [38]:
init_notebook_mode(connected=True)

# scope = ''
scope = ['ND', 'SD', 'NE', 'KS', 'MO', 'IA', 'MN', 'WI', 'IL', 'KY', 'IN', 'MI', 'OH', 'PA', 'WV', 'MD', 'DE',
         'NY', 'TN', 'AR', 'OK', 'VA', 'NC']

def fips_mapping(df, title, variable, treatment, legend_title, linspacing, divergent=False, reverse=False):

    data = pivoted_df[variable][treatment]
    
    # use 'linspacing' parameters to create a bin list, and specify rounding if values are small-ish
    bin_list = np.linspace(linspacing[0], linspacing[1], linspacing[2]).tolist()
    rounding = True
    if linspacing[1] < 10:
        rounding = False

    kwargs = {}
    if scope:
        kwargs['scope'] = scope

    if divergent:
        # convert matplotlib (r, g, b, x) tuple color format to 'rgb(r, g, b)' Plotly string format
        cmap = get_cmap('RdBu')  # or RdYlBu for better differentiation vs. missing data squares in tiling map
        custom_rgb_cmap = [cmap(x) for x in np.linspace(0, 1, (linspacing[2] + 1))]
        custom_plotly_cmap = []
        for code in custom_rgb_cmap:
            plotly_code = 'rgb({},{},{})'.format(code[0] * 255.0, code[1] * 255.0, code[2] * 255.0)
            custom_plotly_cmap.append(plotly_code)
        if reverse:
            custom_plotly_cmap.reverse()

        kwargs['state_outline'] = {'color': 'rgb(100,100,100)', 'width': 1.0}
        kwargs['colorscale'] = custom_plotly_cmap

    fig = ff.create_choropleth(fips=data.index.tolist(),
                               values=data.tolist(),
                               binning_endpoints=bin_list,
                               round_legend_values=rounding,
                               county_outline={'color': 'rgb(255,255,255)', 'width': 0.25},
                               legend_title=legend_title,
                               title=title,
                               paper_bgcolor='rgba(0,0,0,0)',
                               plot_bgcolor='rgba(0,0,0,0)',
                               **kwargs)
    iplot(fig)

In [39]:
fips_mapping(pivoted_df, "Stover yield @ 25% removal rate", 'stover_yield_Mg_ha-1', 'G25S', '(Mg ha-1)', (2, 4, 21))